In [1]:
import os
import requests
import sys

# Get Data

In [3]:
# Get case data.

url = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyCaseCountData.xlsx"
resp = requests.get(url)

output = open('CaseCounts.xlsx', 'wb')
output.write(resp.content)
output.close()

# Get fatality data.

url = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyFatalityCountData.xlsx"
resp = requests.get(url)

output = open('Fatalities.xlsx', 'wb')
output.write(resp.content)
output.close()

# Get estimated active cases data.

url = "https://dshs.texas.gov/coronavirus/TexasCOVID-19ActiveCaseDatabyCounty.xlsx"
resp = requests.get(url)

output = open('EstActiveCases.xlsx', 'wb')
output.write(resp.content)
output.close()

# Get cumulative tests data.

url = "https://dshs.texas.gov/coronavirus/TexasCOVID-19CumulativeTestsOverTimebyCounty.xlsx"
resp = requests.get(url)

output = open('TestCounts.xlsx', 'wb')
output.write(resp.content)
output.close()

# # Get 2020 population estimates data.

# url = "https://demographics.texas.gov/Resources/TPEPP/Projections/2018/2018allcntytot.zip"
# resp = requests.get(url)

# output = open('PopEstimates.xlsx', 'wb')
# output.write(resp.content)
# output.close()

# Clean Data

In [4]:
import numpy as np
import os
import pandas as pd
import sys
import string
import datetime

In [16]:
cases = pd.read_excel("CaseCounts.xlsx", header=2)
cases.head()

,County Name,Cases 03-04,Cases 03-05,Cases 03-06,Cases 03-09,Cases 03-10,Cases 03-11,Cases 03-12,Cases 03-13,Cases 03-15,...,Cases 01-02,Cases 01-03,Cases 01-04,Cases 01-05,Cases 01-06,Cases 01-07,Cases 01-08,Cases 01-09,Cases 01-10,Cases 01-11
0,Anderson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3950.0,3950.0,3950.0,4367.0,4472.0,4471.0,4471.0,4485.0,4508.0,4508.0
1,Andrews,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1433.0,1433.0,1433.0,1438.0,1438.0,1486.0,1486.0,1523.0,1523.0,1523.0
2,Angelina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3622.0,3622.0,3622.0,3692.0,3692.0,3692.0,3790.0,3851.0,3851.0,3851.0
3,Aransas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,563.0,569.0,569.0,577.0,579.0,589.0,596.0,598.0,607.0,607.0
4,Archer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,498.0,498.0,498.0,536.0,544.0,565.0,573.0,574.0,574.0,574.0


In [17]:
# We need to clean up the column names.
for i in range(1, len(cases.columns)):
    col = cases.columns[i]
    test = cases.columns[i]
    test = test.replace(" ", "")
    test = test.replace("\r", "")
    test = test.replace("\n", "")
    test = test.replace("Cases", "")
    test = test.replace("*", "")
    is_2021 = False
    if "01-" in col:
        is_2021 = True
        
    if is_2021:
        test = test + "-2021"
    else:
        test = test + "-2020"
    cases = cases.rename(columns={col:test})

In [20]:
cases = cases.melt(id_vars=["County Name"],
          var_name="Date",
          value_name="Cases")

cases = cases.dropna()

In [18]:
cases.head()

,County Name,03-04-2020,03-05-2020,03-06-2020,03-09-2020,03-10-2020,03-11-2020,03-12-2020,03-13-2020,03-15-2020,...,01-02-2021,01-03-2021,01-04-2021,01-05-2021,01-06-2021,01-07-2021,01-08-2021,01-09-2021,01-10-2021,01-11-2021
0,Anderson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3950.0,3950.0,3950.0,4367.0,4472.0,4471.0,4471.0,4485.0,4508.0,4508.0
1,Andrews,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1433.0,1433.0,1433.0,1438.0,1438.0,1486.0,1486.0,1523.0,1523.0,1523.0
2,Angelina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3622.0,3622.0,3622.0,3692.0,3692.0,3692.0,3790.0,3851.0,3851.0,3851.0
3,Aransas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,563.0,569.0,569.0,577.0,579.0,589.0,596.0,598.0,607.0,607.0
4,Archer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,498.0,498.0,498.0,536.0,544.0,565.0,573.0,574.0,574.0,574.0


In [21]:
# Make datetime instead of string.
cases["Date"] = [datetime.datetime.strptime(d, "%m-%d-%Y") for d in cases["Date"].values]

In [22]:
# Now add additional cases column.
cases["DailyNew"] = "x"

In [23]:
dates = np.sort(cases["Date"].unique())
for i in range(1, len(dates)):
    for county in cases["County Name"].unique():
        term1 = cases[(cases["County Name"] == county)&(cases["Date"] == dates[i])]["Cases"].values[0]
        term2 = cases[(cases["County Name"] == county)&(cases["Date"] == dates[i-1])]["Cases"].values[0]
        cases[(cases["County Name"] == county)&(cases["Date"] == dates[i])]["DailyNew"] = term1 - term2

<ipython-input-23-12dd29b171fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases[(cases["County Name"] == county)&(cases["Date"] == dates[i])]["DailyNew"] = term1 - term2


In [24]:
cases.to_csv("CaseData.csv", index=False)

In [25]:
cases.head()

,County Name,Date,Cases,DailyNew
0,Anderson,2020-03-04,0.0,x
1,Andrews,2020-03-04,0.0,x
2,Angelina,2020-03-04,0.0,x
3,Aransas,2020-03-04,0.0,x
4,Archer,2020-03-04,0.0,x


In [15]:
cases.tail()

,County Name,Date,Cases,DailyNew
82401,Young,2020-01-11,1389.0,x
82402,Zapata,2020-01-11,1077.0,x
82403,Zavala,2020-01-11,1021.0,x
82404,Total,2020-01-11,1730312.0,x
82406,Counties Reporting Cases,2020-01-11,254.0,x
